In [ ]:
from config import load_config
from model import Model

import torch.nn as nn
import torch
import torch.nn.functional as F

In [ ]:
model = Model("/data1/model/mistral-7b-base/")

In [ ]:
logits, nlls = model.forward("Hello, world!")

In [ ]:
logits, nlls, token_ids = model.forward("Hello, world!", return_tokens=True)
print(logits.shape, nlls.shape, token_ids.shape)

print(nlls)

In [ ]:
logits, nlls, token_ids = model.forward("Hello, world!", return_tokens=True)

# Test if the above obtained nlls are equal to the ones obtained by the following code
criterian = nn.NLLLoss(reduction='none')
log_softmax = nn.LogSoftmax(dim=1)

logits = torch.unsqueeze(logits, 0)
# token_ids = torch.unsqueeze(token_ids, 0)
print(logits.shape, token_ids.shape)

In [ ]:
logits = torch.permute(logits, (0, 2, 1)) # B,L,V -> B,V,L # Reason: nn.NLLLoss requires class to be of dimension 1
shift_logits = logits[:, :, :-1]
shift_token_ids = token_ids[..., 1:]

nlls2 = criterian(log_softmax(shift_logits), 
                  shift_token_ids)
print(nlls2)
print(nlls)
# Inconsistent

In [ ]:
# Try to fix nlls
logits, nlls, token_ids = model.forward("Hello, world!", return_tokens=True)

probs = F.softmax(logits, dim=-1) # shape: [L, V]
nlls_cands = torch.zeros(probs.shape[0], dtype=torch.float32)
for i in range(probs.shape[0]-1):
    nlls_cands[i] = -torch.log(probs[i, token_ids[0, i+1]])

print(nlls_cands) # Now, it is fixed